[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openscilabs/isda/blob/main/dtlz.ipynb)

# MISDA Benchmark: DTLZ Suite

This notebook evaluates MISDA's capability in Multi-Objective dimensionality reduction.
Key metrics evaluated:
1.  **Reconstruction Fidelity (SES)**: Linear reconstruction capability (Warning: Penalizes non-linear manifolds).
2.  **Pareto Consistency**: Whether the surrogate preserves the dominance structure (Precision/Recall). Designed for EMO.

In [ ]:
# Install misda from repository
!pip install git+https://github.com/openscilabs/isda.git

import numpy as np
import pandas as pd
import misda
import math

print("Libraries loaded.")
results = {}


In [ ]:
# === MOP Generators (DTLZ) ===

def generate_dtlz2(N=1000, M=3, n_vars=12, on_front=False):
    """
    Generates N samples of DTLZ2 with M objectives.
    """
    rng = np.random.default_rng()
    k = n_vars - M + 1
    X = rng.uniform(0.0, 1.0, size=(N, n_vars))
    if on_front:
        X[:, (M-1):] = 0.5
    xm = X[:, (M-1):] 
    g = np.sum((xm - 0.5)**2, axis=1)
    F = np.zeros((N, M))
    for i in range(M):
        f = (1.0 + g)
        for j in range(M - 1 - i):
            f *= np.cos(X[:, j] * math.pi / 2.0)
        if i > 0:
            f *= np.sin(X[:, M - 1 - i] * math.pi / 2.0)
        F[:, i] = f
    return F, X

def generate_dtlz5(N=1000, M=3, n_vars=12, on_front=False):
    """
    Generates N samples of DTLZ5 (Degenerate curve).
    """
    rng = np.random.default_rng()
    k = n_vars - M + 1
    X = rng.uniform(0.0, 1.0, size=(N, n_vars))
    if on_front:
        X[:, (M-1):] = 0.5
    xm = X[:, (M-1):]
    g = np.sum((xm - 0.5)**2, axis=1)
    theta = np.zeros((N, M-1))
    theta[:, 0] = X[:, 0] * math.pi / 2.0
    gr = g[:, np.newaxis]
    for i in range(1, M-1):
        theta[:, i] = ((math.pi / (4.0 * (1.0 + gr))) * (1.0 + 2.0 * gr * X[:, i][:, np.newaxis])).ravel()
    F = np.zeros((N, M))
    for i in range(M):
        f = (1.0 + g)
        for j in range(M - 1 - i):
            f *= np.cos(theta[:, j])
        if i > 0:
            f *= np.sin(theta[:, M - 1 - i])
        F[:, i] = f
    return F, X

In [ ]:
# Validation Utility: misda.compile_benchmark_summary is used instead.


In [ ]:
# === Visualization Utilities ===
import matplotlib.pyplot as plt

def simple_linear_predict(X_train, y_train, X_test):
    """ Numpy-based Linear Regression to avoid sklearn dependency """
    # Add bias term
    N = X_train.shape[0]
    X_b = np.column_stack([np.ones(N), X_train])
    # Solve (X^T X)^-1 X^T y
    beta = np.linalg.lstsq(X_b, y_train, rcond=None)[0]
    # Predict
    M = X_test.shape[0]
    X_test_b = np.column_stack([np.ones(M), X_test])
    return X_test_b @ beta

def plot_reconstruction_3d(result_obj, df_original, title="Reconstruction"):
    M = df_original.shape[1]
    if M < 3: return
    Y = df_original.values
    indices = result_obj.best_mis['mis_indices']
    X_subset = Y[:, indices]
    
    # Use numpy for prediction
    Y_hat = simple_linear_predict(X_subset, Y, X_subset)
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    if M > 3: title += " (Projected)"
    ax.scatter(Y[:, 0], Y[:, 1], Y[:, 2], c='blue', alpha=0.15, label='Original')
    step = 1 if len(Y) < 500 else 2
    ax.scatter(Y_hat[::step, 0], Y_hat[::step, 1], Y_hat[::step, 2], c='red', marker='x', alpha=0.6, label='Reconstructed')
    ax.set_title(title)
    plt.show()

## 1. DTLZ Test Cases
Running standard suite...

In [ ]:
# DTLZ2 (M=3)
Y, _ = generate_dtlz2(N=500, M=3)
df = pd.DataFrame(Y, columns=['f1', 'f2', 'f3'])
name = "DTLZ2 (M=3)"
res = misda.analyze(df, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 3}}

# DTLZ5 (M=3)
Y, _ = generate_dtlz5(N=500, M=3)
df = pd.DataFrame(Y, columns=['f1', 'f2', 'f3'])
name = "DTLZ5 (M=3)"
res = misda.analyze(df, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 2}}

# DTLZ2 + Redundancy
Y_base, _ = generate_dtlz2(N=500, M=3)
rng = np.random.default_rng(42)
all_feats = []
names = []
for i in range(3):
    orig = Y_base[:, i]
    all_feats.append(orig)
    names.append(f"f{i+1}")
    for k in range(3):
        copy = orig + 0.05 * rng.normal(size=len(orig))
        all_feats.append(copy)
        names.append(f"f{i+1}_k{k}")
df_red = pd.DataFrame(np.column_stack(all_feats), columns=names)
name = "DTLZ2 + Red"
res = misda.analyze(df_red, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df_red, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 3}}


## 2. High-Dimensional (M=10)

In [ ]:
# DTLZ2 (M=10) [Random]
Y, _ = generate_dtlz2(N=500, M=10, n_vars=20)
df = pd.DataFrame(Y, columns=[f'f{i+1}' for i in range(10)])
name = "DTLZ2 (M=10) Rand"
res = misda.analyze(df, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 10}}

# DTLZ5 (M=10) [Random]
Y, _ = generate_dtlz5(N=500, M=10, n_vars=20)
df = pd.DataFrame(Y, columns=[f'f{i+1}' for i in range(10)])
name = "DTLZ5 (M=10) Rand"
res = misda.analyze(df, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 2}}

# DTLZ2 (M=10) [Frontier]
Y, _ = generate_dtlz2(N=500, M=10, n_vars=20, on_front=True)
df = pd.DataFrame(Y, columns=[f'f{i+1}' for i in range(10)])
name = "DTLZ2 (M=10) Opd"
res = misda.analyze(df, caution=0.5, run_ses=True, name=name)
print(res.summary())
plot_reconstruction_3d(res, df, title=name)
results[name] = {"result_obj": res, "truth": {"intrinsic_dim_expected": 10}}

In [ ]:
print("\n=== FINAL MISDA PERFORMANCE (with Pareto Consistency) ===")
df_summary = misda.compile_benchmark_summary(results)
print(df_summary.to_string(index=False))
